### Install prerequisite packages for gym environment and RL algorithms

In [1]:
!pip install -q gym numpy gymnasium stable-baselines3

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [2]:
import carla
import os
import gymnasium as gym
import numpy as np
from gymnasium import spaces
from time import sleep
import queue

In [6]:
class CarlaEnv(gym.Env):
    """
    Custom Environment that follows gym interface.
    This is a env to interface with CARLA simulator
    """

    # Because of jupyter notebook, we cannot implement the GUI ('human' render mode)
    metadata = {"render_modes": ["console"]}
    
    def camera_callback(self, image):
        self.camera1_data = np.array(image.raw_data).reshape(600, 800, 4)[..., [2,1,0,3]]
    
    def lane_callback(self, event):
        self.crossed_lane = True

    def collision_callback(self, event):
        self.collision_detected = True

    def __init__(self, render_mode="console"):
        super(CarlaEnv, self).__init__()
        self.render_mode = render_mode
        # Create array to store camera 1 (rgb_view) data
        self.camera1_data = np.empty((600, 800, 4))
        
        self.crossed_lane = False
        self.collision_detected = False
        
        # Connect to CARLA
        client_name = os.environ.get("CLIENT_NAME", "DOES NOT EXIST")
        if client_name == "DOES NOT EXIST":
            raise Exception("The environment variable for the container name of the carla server has not been set")

        # Connect to the client and retrieve the world object
        client = carla.Client(client_name, 2000)
        self.world = client.get_world()
        self.world.tick()

        # Define action and observation space
        # They must be gym.spaces objects
        # In this case it is buckets for throttle, steering and brake
        n_actions = 462
        self.action_space = spaces.Discrete(n_actions)
        # The observation will be the RGB camera view (for now)
        self.observation_space = spaces.Box(
            low=0, high=255, shape=(600,800,4), dtype=np.uint8
        )
        
        # Get camera listener working
        cameras = []
        lane_sensors = []
        collision_sensors = []
        for actor in self.world.get_actors():
            if(actor.type_id == 'sensor.camera.rgb'):
                cameras.append(actor)
            elif (actor.type_id == 'sensor.other.lane_invasion'):
                lane_sensors.append(actor)
            elif (actor.type_id == 'sensor.other.collision'):
                  collision_sensors.append(actor)
        self.collision_sensor = collision_sensors[0]
        self.lane_sensor = lane_sensors[0]
        self.camera = cameras[0]
        vehicles = []
        for actor in self.world.get_actors():
            if(actor.type_id == 'vehicle.tesla.model3'):
                vehicles.append(actor)
        for vehicle in vehicles:
            if(vehicle.attributes["role_name"] == "ego_vehicle"):
                self.ego_vehicle = vehicle
        self.reset()

    def reset(self, seed=None, options=None):
        """
        Important: the observation must be a numpy array
        :return: (np.array)
        """
        super().reset(seed=seed, options=options)
        
        self.collision_sensor.stop()
        self.lane_sensor.stop()
        self.camera.stop()

        # move vehicle to correct location
        initLocation = carla.Location(x=-51.404804, y=-76.343437, z=0.053576)
        initRotation = carla.Rotation(pitch=0.0, yaw=90.0, roll=0.0)
        self.ego_vehicle.set_transform(carla.Transform(initLocation, initRotation))
        
        vehicle_control_cmd = carla.VehicleControl(0.0, 0.0, 1.0)
        self.ego_vehicle.apply_control(vehicle_control_cmd)
        
        self.crossed_lane = False
        self.collision_detected = False

        self.collision_sensor.listen(self.collision_callback)
        self.lane_sensor.listen(self.lane_callback)
        self.camera.listen(self.camera_callback)

        self.world.wait_for_tick()
        
        return self.camera1_data.astype(np.uint8), {}  # empty info dict

    def step(self, action):
        # steering with throttle 21*11=231 combos
        # steering with brake 21*11=231 combos
        # in total 462 combos
        throttle = 0
        steer = 0
        brake = 0
        if(action < 231): # steering with throttle
            throttle = int(action / 21) / 10.0
            steer = (action % 21) / 10.0 - 1.0
        elif (action < 462): # steering with brake
            action -= 231
            brake = int(action / 21) / 10.0
            steer = (action % 21) / 10.0 - 1.0
        vehicle_control_cmd = carla.VehicleControl(throttle, steer, brake)
        self.ego_vehicle.apply_control(vehicle_control_cmd)
        
        
        # Let the simulator respond
        self.world.wait_for_tick()

        # Optionally we can pass additional info, we are not using that for now
        info = {}
        truncated = False
        reward = 2
        if(self.crossed_lane):
            reward = 0
        terminated = self.collision_detected
        if(terminated):
            self.camera.stop()
            self.lane_sensor.stop()
            reward = 0
        return (
            self.camera1_data.astype(np.uint8),
            reward,
            terminated,
            truncated,
            info,
        )

    def render(self):
        pass

    def close(self):
        pass

In [7]:
env = CarlaEnv()
env.reset()

(array([[[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],
 
        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],
 
        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],
 
        ...,
 
        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],
 
        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],
 
        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         ...,
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]]], dtype=uint8),
 {})

In [9]:
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import DQN

In [11]:
env = CarlaEnv()
# If the environment don't follow the interface, an error will be thrown
check_env(env, warn=True)
env.reset()

/tmp/ipykernel_31/3210272497.py:99: RuntimeWarning: invalid value encountered in cast
  return self.camera1_data.astype(np.uint8), {}  # empty info dict


(array([[[186, 203, 223, 255],
         [186, 203, 223, 255],
         [186, 204, 223, 255],
         ...,
         [ 88,  86,  86, 255],
         [ 88,  86,  88, 255],
         [ 88,  86,  86, 255]],
 
        [[186, 203, 223, 255],
         [186, 204, 223, 255],
         [186, 204, 223, 255],
         ...,
         [131, 129, 129, 255],
         [131, 129, 129, 255],
         [131, 129, 129, 255]],
 
        [[186, 204, 223, 255],
         [186, 203, 223, 255],
         [187, 204, 223, 255],
         ...,
         [140, 139, 138, 255],
         [142, 139, 139, 255],
         [142, 140, 140, 255]],
 
        ...,
 
        [[224, 220, 219, 255],
         [224, 221, 221, 255],
         [222, 218, 215, 255],
         ...,
         [222, 220, 215, 255],
         [222, 219, 217, 255],
         [221, 218, 215, 255]],
 
        [[223, 220, 218, 255],
         [224, 221, 220, 255],
         [224, 220, 219, 255],
         ...,
         [220, 216, 212, 255],
         [220, 217, 214, 255],
    

In [12]:
model = DQN("MlpPolicy", env, verbose=1, buffer_size=1000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [ ]:
model.learn(1000)